In [6]:
import pathlib
import sys
sys.path.append(str(pathlib.Path.cwd().absolute().parent))

import polars as pl
import altair as alt

from db_utils import CACHED_DF_PATH, CACHE_VERSION

In [13]:
pl.read_parquet("//allen/programs/mindscope/workgroups/dynamicrouting/ben/spike_counts.parquet")


trial_index,unit_id,baseline,response
i64,str,i64,i64
0,"""626791_2022-08-16_A-22""",0,0
1,"""626791_2022-08-16_A-22""",0,1
2,"""626791_2022-08-16_A-22""",0,1
3,"""626791_2022-08-16_A-22""",0,0
4,"""626791_2022-08-16_A-22""",2,0
…,…,…,…
533,"""742903_2024-10-22_D-180""",0,0
534,"""742903_2024-10-22_D-180""",0,0
535,"""742903_2024-10-22_D-180""",0,0


In [61]:
df = (
    pl.scan_parquet("//allen/programs/mindscope/workgroups/dynamicrouting/ben/lda.parquet")
    .filter(
        pl.col('lda').is_not_nan(),
        # pl.col('lda').is_between(-0.451, -0.449),
        pl.col('aud_response_r2').ge(0.6),
    )
    .sort('aud_response_r2', descending=False)
    .slice(0, 3)
    .join(
        (
            pl.scan_parquet("//allen/programs/mindscope/workgroups/dynamicrouting/ben/spike_counts.parquet")
        ),
        on='unit_id',
        how='inner',
    )
    .with_columns(
        session_id=pl.col('unit_id').str.split('_').list.slice(0, 2).list.join('_'),
    )
    .join(
        pl.scan_parquet(CACHED_DF_PATH.format(CACHE_VERSION, 'trials')).select('session_id', 'start_time', 'trial_index', 'is_vis_stim'),
        on=['session_id', 'trial_index'],
        how='inner',
    )
).collect()
display(df)

base = (
    alt.Chart(df)
    .mark_point()
    .encode(
        x='start_time',
        color='is_vis_stim',
        row='unit_id',
    )
)
alt.hconcat(
    base.encode(
        y='baseline',
    ),
    base.encode(
        y='response',
    )#.properties(title=f'r^2 aud={df["aud_response_r2"][0]:.2f}, vis={df["vis_response_r2"][0]:.2f} | lda={df["lda"][0]:.2f}'),
).resolve_scale(
    y='shared',
    x='shared',
)


unit_id,drift_rating,vis_response_r2,aud_response_r2,presence_ratio,lda,trial_index,baseline,response,session_id,start_time,is_vis_stim
str,i32,f64,f64,f64,f64,i64,i64,i64,str,f64,bool
"""644866_2023-02-08_C-415""",1,0.458911,0.607529,0.988095,0.104724,0,0,0,"""644866_2023-02-08""",1353.21643,false
"""644866_2023-02-08_C-415""",1,0.458911,0.607529,0.988095,0.104724,1,0,0,"""644866_2023-02-08""",1360.6894,false
"""644866_2023-02-08_C-415""",1,0.458911,0.607529,0.988095,0.104724,2,0,0,"""644866_2023-02-08""",1366.49432,false
"""644866_2023-02-08_C-415""",1,0.458911,0.607529,0.988095,0.104724,3,2,0,"""644866_2023-02-08""",1372.96633,false
"""644866_2023-02-08_C-415""",1,0.458911,0.607529,0.988095,0.104724,4,0,1,"""644866_2023-02-08""",1381.64018,false
…,…,…,…,…,…,…,…,…,…,…,…
"""742903_2024-10-21_E-283""",1,0.255507,0.603114,1.0,-0.023985,505,0,0,"""742903_2024-10-21""",6131.3185,false
"""742903_2024-10-21_E-283""",1,0.255507,0.603114,1.0,-0.023985,506,0,1,"""742903_2024-10-21""",6136.95655,true
"""742903_2024-10-21_E-283""",1,0.255507,0.603114,1.0,-0.023985,507,0,2,"""742903_2024-10-21""",6143.04501,true


alt.HConcatChart(...)